In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
import time
from sklearn.metrics import accuracy_score,  f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score
import random
import time
import collections
import warnings
%matplotlib inline

C:\Users\RUSHI\Anaconda Python\envs\tf-gpu\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\RUSHI\Anaconda Python\envs\tf-gpu\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#depre

In [2]:
train_tcga_df = pd.read_csv('D://GitHub/Gene-compression-and-Cancer-type-classification/data/train_tcga_expression_matrix_processed.tsv', header=0, sep='\t')
test_tcga_df = pd.read_csv('D://GitHub/Gene-compression-and-Cancer-type-classification/data/test_tcga_expression_matrix_processed.tsv', header=0, sep='\t')

labels_tcga_df = pd.read_csv('D://GitHub/Gene-compression-and-Cancer-type-classification/data/tcga_sample_identifiers.tsv', header=0, sep='\t')

In [3]:
train_tcga_df.head()

,sample_id,1,10,100,1000,10000,10001,10002,10003,100037417,...,9987,9988,9989,999,9990,9991,9992,9993,9994,9997
0,TCGA-LL-A73Z-01,202.0,28.5,329.0,84.5,492.0,448.0,4.590,14.70,337.0,...,3430.0,717.0,1800.0,6360.0,299.0,2310.0,10.60,3190.0,337.0,892.0
1,TCGA-55-8207-01,77.5,22.5,74.5,13.1,784.0,333.0,2.540,176.00,153.0,...,6050.0,923.0,2490.0,11300.0,1150.0,4030.0,9.08,2890.0,316.0,301.0
2,TCGA-FF-A7CR-01,152.0,0.0,3020.0,26.6,486.0,497.0,0.000,8.47,348.0,...,4930.0,897.0,861.0,39.7,464.0,3320.0,0.00,1330.0,606.0,558.0
3,TCGA-BK-A13C-11,80.5,40.0,70.6,284.0,2420.0,325.0,1.200,91.40,231.0,...,3890.0,737.0,1410.0,10.9,1120.0,1990.0,5.24,3090.0,673.0,263.0
4,TCGA-EB-A6L9-06,319.0,0.0,422.0,184.0,423.0,392.0,0.945,2.36,585.0,...,1930.0,328.0,1340.0,7010.0,450.0,563.0,10.90,3780.0,37.3,1120.0


In [4]:
test_tcga_df.head()

,sample_id,1,10,100,1000,10000,10001,10002,10003,100037417,...,9987,9988,9989,999,9990,9991,9992,9993,9994,9997
0,TCGA-CN-5365-01,70.5,0.000,1310.0,18.70,73.9,757.0,1.82,10.50,414.0,...,2730.0,612.0,5780.0,10400.0,879.0,3830.0,4.10,4220.0,308.0,1300.0
1,TCGA-LP-A7HU-01,27.8,0.949,517.0,6.17,63.6,365.0,4.27,9.02,633.0,...,4510.0,543.0,1360.0,4960.0,510.0,2220.0,17.60,2410.0,233.0,993.0
2,TCGA-22-5491-11,87.6,3.760,88.3,14.40,642.0,295.0,6.84,59.90,159.0,...,4420.0,479.0,2190.0,4480.0,870.0,2120.0,5.13,5030.0,285.0,530.0
3,TCGA-CS-6667-01,75.8,0.000,55.6,2720.00,2170.0,281.0,3.90,9.57,251.0,...,5460.0,1550.0,1370.0,12.8,1430.0,601.0,23.70,7530.0,473.0,258.0
4,TCGA-20-1684-01,63.1,0.703,75.2,4500.00,792.0,433.0,1.64,4.16,623.0,...,5200.0,812.0,1070.0,8410.0,505.0,1170.0,14.80,1910.0,281.0,221.0


In [5]:
labels_tcga_df.head()

,sample_id,sample_type,cancer_type
0,TCGA-02-0047-01,Primary Solid Tumor,GBM
1,TCGA-02-0055-01,Primary Solid Tumor,GBM
2,TCGA-02-2483-01,Primary Solid Tumor,GBM
3,TCGA-02-2485-01,Primary Solid Tumor,GBM
4,TCGA-02-2486-01,Primary Solid Tumor,GBM


In [6]:
train_id = train_tcga_df['sample_id']
test_id = test_tcga_df['sample_id']
label_id = labels_tcga_df['sample_id']

In [7]:
merge_train = []
merge_test = []

for i in train_id:
    val = labels_tcga_df.loc[labels_tcga_df['sample_id'] == i]
    merge_train.append(str(val['cancer_type']).split()[1])
    
for i in test_id:
    val = labels_tcga_df.loc[labels_tcga_df['sample_id'] == i]
    merge_test.append(str(val['cancer_type']).split()[1])

In [8]:
train_tcga_df['cancer_type'] = merge_train
train_tcga_df.head()

,sample_id,1,10,100,1000,10000,10001,10002,10003,100037417,...,9988,9989,999,9990,9991,9992,9993,9994,9997,cancer_type
0,TCGA-LL-A73Z-01,202.0,28.5,329.0,84.5,492.0,448.0,4.590,14.70,337.0,...,717.0,1800.0,6360.0,299.0,2310.0,10.60,3190.0,337.0,892.0,BRCA
1,TCGA-55-8207-01,77.5,22.5,74.5,13.1,784.0,333.0,2.540,176.00,153.0,...,923.0,2490.0,11300.0,1150.0,4030.0,9.08,2890.0,316.0,301.0,LUAD
2,TCGA-FF-A7CR-01,152.0,0.0,3020.0,26.6,486.0,497.0,0.000,8.47,348.0,...,897.0,861.0,39.7,464.0,3320.0,0.00,1330.0,606.0,558.0,DLBC
3,TCGA-BK-A13C-11,80.5,40.0,70.6,284.0,2420.0,325.0,1.200,91.40,231.0,...,737.0,1410.0,10.9,1120.0,1990.0,5.24,3090.0,673.0,263.0,UCEC
4,TCGA-EB-A6L9-06,319.0,0.0,422.0,184.0,423.0,392.0,0.945,2.36,585.0,...,328.0,1340.0,7010.0,450.0,563.0,10.90,3780.0,37.3,1120.0,SKCM


In [9]:
test_tcga_df['cancer_type'] = merge_test
test_tcga_df.head()

,sample_id,1,10,100,1000,10000,10001,10002,10003,100037417,...,9988,9989,999,9990,9991,9992,9993,9994,9997,cancer_type
0,TCGA-CN-5365-01,70.5,0.000,1310.0,18.70,73.9,757.0,1.82,10.50,414.0,...,612.0,5780.0,10400.0,879.0,3830.0,4.10,4220.0,308.0,1300.0,HNSC
1,TCGA-LP-A7HU-01,27.8,0.949,517.0,6.17,63.6,365.0,4.27,9.02,633.0,...,543.0,1360.0,4960.0,510.0,2220.0,17.60,2410.0,233.0,993.0,CESC
2,TCGA-22-5491-11,87.6,3.760,88.3,14.40,642.0,295.0,6.84,59.90,159.0,...,479.0,2190.0,4480.0,870.0,2120.0,5.13,5030.0,285.0,530.0,LUSC
3,TCGA-CS-6667-01,75.8,0.000,55.6,2720.00,2170.0,281.0,3.90,9.57,251.0,...,1550.0,1370.0,12.8,1430.0,601.0,23.70,7530.0,473.0,258.0,LGG
4,TCGA-20-1684-01,63.1,0.703,75.2,4500.00,792.0,433.0,1.64,4.16,623.0,...,812.0,1070.0,8410.0,505.0,1170.0,14.80,1910.0,281.0,221.0,OV


In [10]:
def returnDf2(c_type):
    df2  = train_tcga_df.drop(['sample_id'], axis=1)

    fraction = collections.Counter(merge_train)[c_type]/len(merge_train)

    df2_0 = df2[df2.cancer_type!=c_type]
    df2_0.loc[:,"cancer_type"] = 0
    df2_0 = df2_0.sample(frac = fraction)

    df2_1 = df2[df2.cancer_type==c_type]
    df2_1.loc[:,"cancer_type"] = 1

    df2 = pd.concat([df2_0, df2_1])
    df2["cancer_type"] = df2["cancer_type"].astype('int')

    return df2

In [11]:
def returnTest2(c_type):
    test2  = test_tcga_df.drop(['sample_id'], axis=1)

    fraction = collections.Counter(merge_test)[c_type]/len(merge_test)

    test2_0 = test2[test2.cancer_type!=c_type]
    test2_0.loc[:,"cancer_type"] = 0
    test2_0 = test2_0.sample(frac = fraction)

    test2_1 = test2[test2.cancer_type==c_type]
    test2_1.loc[:,"cancer_type"] = 1

    test2 = pd.concat([test2_0, test2_1])
    test2 = test2.astype('int')
    return test2

In [12]:
def runSVM(df2, test2):
    #first Support vector machine classifier with 16149 variables
    time_laps = []
    for i in range(4):
        start_time = time.time()
        clf = svm.SVC(kernel='linear')
        clf.fit(df2.iloc[:,0:df2.shape[1]],df2["cancer_type"])
        pred = clf.predict(test2.iloc[:,0:test2.shape[1]])
        accuracy = accuracy_score(test2.cancer_type,pred)
        laps = time.time() - start_time
        time_laps.append(laps)
    avg_time_laps = np.mean(time_laps)
    SVM_accuracy = accuracy
    SVM_computation_time = avg_time_laps
    return SVM_accuracy, SVM_computation_time

In [13]:
def returnPCA(df2, test2):
    warnings.filterwarnings("ignore")

    start_time = time.time()
    # Normalizing the full dataframe
    new = pd.concat([df2,test2],axis=0)
    standard = StandardScaler()
    result = standard.fit_transform(new.iloc[:,0:new.shape[1]])
    result = pd.DataFrame(result)

    # PCA calculation
    pca = PCA(n_components=min(800, result.shape[0], result.shape[1]))
    principal_comp = pca.fit_transform(result)
    pca_time = time.time()-start_time


    principal_comp = pd.DataFrame(principal_comp)
    explanation = pca.explained_variance_ratio_
    cummulative_sum = explanation.cumsum()

#     print("Time to compute PCA {}".format(pca_time))


#     plt.figure(figsize=(14,6))
#     plt.plot(cummulative_sum*100,linewidth=4)
#     plt.xlabel("Number of Variables in Use",size=15,color="darkred")
#     plt.ylabel("Variability Explained in %",size=15,color="darkred")
#     plt.title("Almost 90% Variablity explained by 800 Variables",size=20,color="darkred")
    return principal_comp, explanation, cummulative_sum, pca_time

In [14]:
def prepPCAClassifier(df2, test2, principal_comp):
    y_col = pd.concat([df2.cancer_type,test2.cancer_type],axis=0)
    principal_comp = pd.DataFrame(principal_comp)
    principal_comp = pd.concat([principal_comp,y_col.reset_index()],axis=1).drop("index",axis=1)
    train_x,test_x,train_y,test_y = train_test_split(principal_comp.iloc[:,0:principal_comp.shape[1]],principal_comp.cancer_type,test_size=0.25)
    return train_x,test_x,train_y,test_y

In [15]:
# SVM model after PCA

def PCAClassifier(train_x, train_y, test_x, test_y):
    com_time_laps=[]
    for i in range(4):
        start_time = time.time()
        clf2 = svm.SVC()
        clf2.fit(train_x,train_y)
        accuracy = accuracy_score(clf2.predict(test_x),test_y) * 100
        accuracy=round(accuracy)
        time_laps_after_pca = time.time() - start_time
        com_time_laps.append(time_laps_after_pca)
#     print("The accuracy is {} and computaiton time is {}".format(accuracy,np.mean(com_time_laps)))
        lis = [accuracy,np.mean(com_time_laps)]
    return accuracy, np.mean(com_time_laps)

In [16]:
c_type_list = collections.Counter(merge_train)
# c_type_list = ['LUAD', 'LGG', 'PRAD', 'STAD']
#c_type_list = ['DLBC']
cols = ['cancer_type', 'Full-data acc.', 'Full-data comp. time', 'PCA components', 'PCA calc. time', 'PCA explained variance', 'PCA acc.', 'PCA comp. time']

results_df = pd.DataFrame(columns = cols)

i = 0
for c_type in c_type_list:
    print(str(i) + ", " + c_type)
    df2 = returnDf2(c_type)
    test2 = returnTest2(c_type)
    accuracy, avg_time_laps = runSVM(df2, test2)
    principal_comp, explanation, cummulative_sum, pca_time = returnPCA(df2, test2)
    train_x,test_x,train_y,test_y = prepPCAClassifier(df2, test2, principal_comp)
    pca_acc, pca_time = PCAClassifier(train_x, train_y, test_x, test_y)
    results_df.loc[i] = [c_type, accuracy,avg_time_laps, len(principal_comp),pca_time, cummulative_sum[len(cummulative_sum)-1], pca_acc, pca_time]
    i = i+1

0, BRCA


C:\Users\RUSHI\Anaconda Python\envs\tf-gpu\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


1, LUAD
2, DLBC
3, UCEC
4, SKCM
5, PRAD
6, HNSC
7, KIRP
8, CESC
9, THCA
10, KIRC
11, STAD
12, COAD
13, READ
14, LGG
15, MESO
16, LAML
17, BLCA
18, OV
19, LUSC
20, ACC
21, THYM
22, ESCA
23, PAAD
24, LIHC
25, SARC
26, GBM
27, TGCT
28, KICH
29, PCPG
30, UCS
31, UVM
32, CHOL


In [19]:
results_df

,cancer_type,Full-data acc.,Full-data comp. time,PCA components,PCA calc. time,PCA explained variance,PCA acc.,PCA comp. time
0,BRCA,0.987013,15.403983,2302,5.157214,0.900279,62,5.157214
1,LUAD,0.990991,7.106756,1122,1.233206,0.975676,56,1.233206
2,DLBC,1.000000,0.112449,96,0.003989,1.000000,42,0.003989
3,UCEC,0.954128,5.585814,1105,1.148181,0.975556,49,1.148181
4,SKCM,1.000000,3.166036,928,0.803602,0.989152,57,0.803602
5,PRAD,1.000000,2.962580,1072,1.071885,0.989382,72,1.071885
6,HNSC,0.972477,5.811713,1103,1.144942,0.977746,54,1.144942
7,KIRP,0.953846,1.731621,637,0.289976,1.000000,49,0.289976
8,CESC,0.967213,3.081010,611,0.253323,1.000000,50,0.253323
9,THCA,1.000000,1.476553,1115,1.148181,0.991581,75,1.148181


EXTRA ANALYSIS for Cancer type BCGA

In [18]:
acc = []
acc_nb = []
f1_svm = []
for i in range(400,1200,50):
    pca = PCA(n_components=i)
    principal_comp = pca.fit_transform(result)
    principal_comp = pd.DataFrame(principal_comp)
    y_col = pd.concat([df2.cancer_type,test2.cancer_type],axis=0)
    principal_comp = pd.DataFrame(principal_comp)
    principal_comp = pd.concat([principal_comp,y_col.reset_index()],axis=1).drop("index",axis=1)

    #SVM
    train_x,test_x,train_y,test_y = train_test_split(principal_comp.iloc[:,0:(principal_comp.shape[1]-1)],principal_comp.cancer_type,test_size=0.45)
    
    clf2 = svm.SVC(kernel='linear',gamma="auto",C=0.001)
    clf2.fit(train_x,train_y)
    a = accuracy_score(clf2.predict(test_x),test_y) * 100
    
    f1_ = f1_score(test_y, clf2.predict(test_x))
    f1_svm.append(f1_)
    acc.append(a)
    
    

NameError: name 'result' is not defined

In [ ]:
plt.figure(figsize=(15,6))
plt.plot([i for i in range(400,1200,50)],acc)
plt.xlabel("Number of Features Selected",size=15)
plt.ylabel("Accuracy in Percentage %",size=15)
plt.title("SVM accuracy as Feature Grows",size=25)
plt.style.use("bmh")

VAE for compression

In [83]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Input, Lambda, Layer, Activation
from tensorflow.keras.layers import Conv2D, Flatten, Lambda
from tensorflow.keras.layers import Reshape, Conv2DTranspose
from tensorflow.keras import backend as K
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import metrics, optimizers

In [99]:
class DataGenerator(tensorflow.keras.utils.Sequence):
    'Generates data for Keras'

    def __init__(self, data_file, chunk_size, num_samples):
        'Initialization'
        self.chunk_size = chunk_size
        self.data_file = data_file
        self.num_samples = num_samples
        self.data = pd.read_table(
            self.data_file,
            chunksize=self.chunk_size,
            index_col=0)

    # Reinitialize (open) data file to cycle through for the next epoch
    def _reinitialize_data(self):
        #print("I reinitalized the data for you")
        self.data = pd.read_table(
            self.data_file,
            chunksize=self.chunk_size,
            index_col=0)

    # Each call requests a batch index between 0 and the total number of batches
    def __len__(self):
        'Denotes the number of batches per epoch'
        num_chunks = int(np.floor(self.num_samples / self.chunk_size))
        return num_chunks

    # Returns a batch of the data
    def __getitem__(self, index):
        try:
            while True:
                data_chunk = self.data.__next__()

                return (data_chunk, None)
        except StopIteration:
            pass

    # Triggered once at the very beginning as well as at the end of each epoch
    # Shuffling the order in which examples are fed to the classifier is helpful
    # so that batches between epochs do not look alike. Doing so will eventually make our model more robust.
    def on_epoch_end(self):
        self._reinitialize_data()

In [77]:
class CustomVariationalLayer(Layer):
    """
    Define a custom layer that learns and performs the training
    """

    def __init__(self, original_dim, z_log_var_encoded,
                 z_mean_encoded, beta, **kwargs):
        # https://keras.io/layers/writing-your-own-keras-layers/
        self.is_placeholder = True
        self.original_dim = original_dim
        self.z_log_var_encoded = z_log_var_encoded
        self.z_mean_encoded = z_mean_encoded
        self.beta = beta

        super(CustomVariationalLayer, self).__init__(**kwargs)

    def vae_loss(self, x_input, x_decoded):
        reconstruction_loss = self.original_dim * \
            metrics.binary_crossentropy(x_input, x_decoded)
        kl_loss = - 0.5 * K.sum(1 + self.z_log_var_encoded -
                                K.square(self.z_mean_encoded) -
                                K.exp(self.z_log_var_encoded), axis=-1)
        return K.mean(reconstruction_loss + (K.get_value(self.beta) * kl_loss))

    def call(self, inputs):
        x = inputs[0]
        x_decoded = inputs[1]
        loss = self.vae_loss(x, x_decoded)
        self.add_loss(loss, inputs=inputs)
        # We won't actually use the output.
        return x

In [78]:
latent_dim = 800

def sampling_maker(epsilon_std):
    def sampling(args):
        # Function with args required for Keras Lambda function
        z_mean, z_log_var = args

        # Draw epsilon of the same shape from a standard normal distribution
        epsilon = K.random_normal(shape=tf.shape(z_mean), mean=0.,
                                  stddev=epsilon_std)

        # The latent vector is non-deterministic and differentiable
        # in respect to z_mean and z_log_var
        z = z_mean + K.exp(z_log_var / 2) * epsilon
        return z
    return sampling

In [79]:
# Initialize parameters
learning_rate = 0.00001
epochs = 10
kappa = 0.01

intermediate_dim = 1600
latent_dim = 800
epsilon_std = 1.0
beta = K.variable(0)

chunk_size = 1000

In [80]:
num_train_samples = train_tcga_df.shape[0]
num_genes = train_tcga_df.shape[1]-2
num_val_samples = test_tcga_df.shape[0]

gene_seq_input = Input(shape=(num_genes,))
hidden_dense_linear = Dense(intermediate_dim, kernel_initializer='glorot_uniform')(gene_seq_input)
hidden_dense_batchnorm = BatchNormalization()(hidden_dense_linear)
hidden_encoded = Activation('relu')(hidden_dense_batchnorm)

# Encoder
z_mean_dense_linear = Dense(latent_dim, kernel_initializer='glorot_uniform')(hidden_encoded)
z_mean_dense_batchnorm = BatchNormalization()(z_mean_dense_linear)
z_mean_encoded = Activation('relu')(z_mean_dense_batchnorm)

z_log_var_dense_linear = Dense(latent_dim, kernel_initializer='glorot_uniform')(gene_seq_input)
z_log_var_dense_batchnorm = BatchNormalization()(z_log_var_dense_linear)
z_log_var_encoded = Activation('relu')(z_log_var_dense_batchnorm)

z = Lambda(sampling_maker(epsilon_std), output_shape=(latent_dim, ))([z_mean_encoded, z_log_var_encoded])

# Decoder
decoder_model = Sequential()
decoder_model.add(Dense(intermediate_dim, activation='relu', input_dim=latent_dim))
decoder_model.add(Dense(num_genes, activation='sigmoid'))
gene_seq_reconstruct = decoder_model(z)


# Connections
adam = optimizers.Adam(lr=learning_rate)
vae_layer = CustomVariationalLayer(num_genes, z_log_var_encoded, z_mean_encoded, beta)([gene_seq_input, gene_seq_reconstruct])
vae = Model(gene_seq_input, vae_layer)
vae.compile(optimizer=adam, loss=None, loss_weights=[beta])

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: expected exactly one node node, found [<gast.gast.FunctionDef object at 0x000001C5771DC108>, <gast.gast.Return object at 0x000001C5771DC2C8>]
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: expected exactly one node node, found [<gast.gast.FunctionDef object at 0x000001C5771DC108>, <gast.gast.Return object at 0x000001C5771DC2C8>]


In [100]:
train_np = train_tcga_df.drop(['sample_id', 'cancer_type'], axis=1)#.to_numpy().astype(np.float)
test_np = test_tcga_df.drop(['sample_id', 'cancer_type'], axis=1)#.to_numpy().astype(np.float)

In [ ]:
train_path = 'D://GitHub/Gene-compression-and-Cancer-type-classification/data/generator_train_tcga_expression_matrix_processed.csv'
test_path = 'D://GitHub/Gene-compression-and-Cancer-type-classification/data/generator_test_tcga_expression_matrix_processed.csv'

train_np.to_csv(train_path, sep='\t')
test_np.to_csv(test_path, sep='\t')

train_gen = DataGenerator(train_path, chunk_size, num_val_samples)
test_gen = DataGenerator(test_path, chunk_size, num_val_samples)

In [95]:
hist = vae.fit_generator(generator=train_gen,
                         validation_data=test_gen,
                         shuffle=True,
                         epochs=epochs)

Train for 1 steps, validate for 1 steps
Epoch 1/10
1/1 [==============================] - 2s 2s/step


_SymbolicException: Inputs to eager execution function cannot be Keras symbolic tensors, but found [<tf.Tensor 'activation_41/Identity:0' shape=(None, 800) dtype=float32>, <tf.Tensor 'activation_40/Identity:0' shape=(None, 800) dtype=float32>]